# Explorando as Tabelas Delta

Este notebook permite explorar e manipular as tabelas Delta do projeto de diário de bordo.

## 1. Configuração do SparkSession

Configurando a sessão Spark com suporte ao Delta Lake.

In [ ]:
from pyspark.sql import SparkSession
import os

# Configuração do SparkSession com Delta Lake
builder = (
    SparkSession.builder
    .appName("ExplorarTabelas")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")
    .config("spark.sql.warehouse.dir", "/app/spark-warehouse")
    .config("spark.driver.extraJavaOptions", "-Dio.delta.sql.DeltaSparkSessionExtension=true")
)

spark = builder.getOrCreate()
print("SparkSession inicializada com sucesso!")

# Verificar se o Delta Lake está disponível
try:
    spark.sql("CREATE TABLE IF NOT EXISTS test_delta USING delta AS SELECT 1 as id")
    spark.sql("DROP TABLE test_delta")
    print("Delta Lake configurado com sucesso!")
except Exception as e:
    print(f"Erro ao verificar Delta Lake: {e}")

25/06/11 03:28:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
SparkSession inicializada com sucesso!
Delta Lake configurado com sucesso!


## 2. Carregando a Tabela Bronze

Lendo os dados da tabela bronze diretamente do formato Delta.

In [14]:
# Carregando a tabela Bronze e registrando no catálogo
bronze_df = spark.read.format("delta").load("/app/data/bronze/b_info_transportes")
bronze_df.createOrReplaceTempView("b_info_transportes")
print("Schema da tabela Bronze:")
bronze_df.printSchema()

print("\nAmostra dos dados:")
bronze_df.show(5)

Schema da tabela Bronze:
root
 |-- DATA_INICIO: string (nullable = true)
 |-- DATA_FIM: string (nullable = true)
 |-- CATEGORIA: string (nullable = true)
 |-- LOCAL_INICIO: string (nullable = true)
 |-- LOCAL_FIM: string (nullable = true)
 |-- DISTANCIA: string (nullable = true)
 |-- PROPOSITO: string (nullable = true)


Amostra dos dados:
+----------------+----------------+---------+------------+---------------+---------+-----------------+
|     DATA_INICIO|        DATA_FIM|CATEGORIA|LOCAL_INICIO|      LOCAL_FIM|DISTANCIA|        PROPOSITO|
+----------------+----------------+---------+------------+---------------+---------+-----------------+
|01-01-2016 21:11|01-01-2016 21:17|  Negocio| Fort Pierce|    Fort Pierce|       51|      Alimentação|
|01-02-2016 01:25|01-02-2016 01:37|  Negocio| Fort Pierce|    Fort Pierce|        5|             null|
|01-02-2016 20:25|01-02-2016 20:38|  Negocio| Fort Pierce|    Fort Pierce|       48|         Entregas|
|01-05-2016 17:31|01-05-2016 17:45|  Neg

## 3. Carregando a Tabela Silver

Lendo os dados da tabela silver processada.

In [15]:
# Carregando a tabela Silver e registrando no catálogo
silver_df = spark.read.format("delta").load("/app/data/silver/s_info_transportes")
silver_df.createOrReplaceTempView("s_info_transportes")
print("Schema da tabela Silver:")
silver_df.printSchema()

print("\nAmostra dos dados:")
silver_df.show(5)

Schema da tabela Silver:
root
 |-- DATA_INICIO: string (nullable = true)
 |-- DATA_FIM: string (nullable = true)
 |-- CATEGORIA: string (nullable = true)
 |-- LOCAL_INICIO: string (nullable = true)
 |-- LOCAL_FIM: string (nullable = true)
 |-- DISTANCIA: string (nullable = true)
 |-- PROPOSITO: string (nullable = true)
 |-- DT_REFE: date (nullable = true)


Amostra dos dados:
+----------------+----------------+---------+----------------+----------+---------+-----------------+----------+
|     DATA_INICIO|        DATA_FIM|CATEGORIA|    LOCAL_INICIO| LOCAL_FIM|DISTANCIA|        PROPOSITO|   DT_REFE|
+----------------+----------------+---------+----------------+----------+---------+-----------------+----------+
|04-01-2016 13:43|04-01-2016 14:01|  negocio|       Kissimmee| Kissimmee|       11|          reunião|2016-01-04|
|04-01-2016 14:36|04-01-2016 15:24|  negocio|       Kissimmee|   Orlando|      155|visita ao cliente|2016-01-04|
|04-01-2016 16:01|04-01-2016 16:49|  negocio|         Or

## 4. Carregando a Tabela Gold

Lendo os dados agregados da tabela gold.

In [16]:
# Carregando a tabela Gold e registrando no catálogo
gold_df = spark.read.format("delta").load("/app/data/gold/info_corridas_do_dia")
gold_df.createOrReplaceTempView("info_corridas_do_dia")
print("Schema da tabela Gold:")
gold_df.printSchema()

print("\nAmostra dos dados:")
gold_df.show(5)

Schema da tabela Gold:
root
 |-- DT_REFE: date (nullable = true)
 |-- QT_CORR: long (nullable = true)
 |-- QT_CORR_NEG: long (nullable = true)
 |-- QT_CORR_PESS: long (nullable = true)
 |-- VL_MAX_DIST: string (nullable = true)
 |-- VL_MIN_DIST: string (nullable = true)
 |-- VL_AVG_DIST: double (nullable = true)
 |-- QT_CORR_REUNI: long (nullable = true)
 |-- QT_CORR_NAO_REUNI: long (nullable = true)


Amostra dos dados:
+----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
|   DT_REFE|QT_CORR|QT_CORR_NEG|QT_CORR_PESS|VL_MAX_DIST|VL_MIN_DIST|VL_AVG_DIST|QT_CORR_REUNI|QT_CORR_NAO_REUNI|
+----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
|2016-06-09|      1|          0|           0|        691|        691|      691.0|            0|                0|
|2016-05-05|      3|          0|           0|         29|        129|      100.0|            1|                2|
|2016

## 5. Manipulação dos Dados

Exemplos de operações que você pode fazer com os dados:

In [17]:
# Exemplo 1: Contagem de registros por tabela
print("Quantidade de registros:")
print(f"Bronze: {bronze_df.count():,}")
print(f"Silver: {silver_df.count():,}")
print(f"Gold: {gold_df.count():,}")

Quantidade de registros:
Bronze: 1,153
Silver: 420
Gold: 114


## Análise das Partições

Vamos explorar as partições da tabela Gold por data de referência:

In [ ]:
# Método 1: Usando os.listdir para ver as partições no sistema de arquivos
import os

partitions = os.listdir('/app/data/gold/info_corridas_do_dia')
partitions = [p for p in partitions if p.startswith('DT_REFE=')]
print(f'Total de partições: {len(partitions)}')
print('\nPrimeiras 10 partições:')
for p in sorted(partitions)[:10]:
    print(p)

In [ ]:
# Método 2: Usando Spark SQL para analisar as partições
print('\nAnálise das partições via Spark SQL:')
gold_df.select('DT_REFE').distinct().orderBy('DT_REFE').show(10)

# Estatísticas por partição
print('\nEstatísticas por partição:')
gold_df.groupBy('DT_REFE') \
    .agg({'QTD_CORRIDAS': 'sum'}) \
    .orderBy('DT_REFE') \
    .withColumnRenamed('sum(QTD_CORRIDAS)', 'TOTAL_CORRIDAS') \
    .show(10)

In [ ]:
# Método 3: Usando SHOW PARTITIONS do Spark SQL
print('\nUsando SHOW PARTITIONS na tabela Gold:')

# Agora podemos usar SHOW PARTITIONS na tabela registrada
spark.sql("SHOW PARTITIONS info_corridas_do_dia").show(20, False)

In [19]:
# Método 4: Usando spark.table() na tabela registrada
print('\nUsando spark.table() diretamente:')

# Agora podemos usar spark.table() porque a tabela está registrada
table_df = spark.table("info_corridas_do_dia")

# Mostrando as partições
print("\nPartições da tabela:")
table_df.select('DT_REFE').distinct().orderBy('DT_REFE').show(20)


Usando spark.table() diretamente:

Partições da tabela:


+----------+
|   DT_REFE|
+----------+
|2016-01-01|
|2016-01-02|
|2016-01-03|
|2016-01-04|
|2016-01-05|
|2016-01-06|
|2016-01-07|
|2016-01-08|
|2016-01-09|
|2016-01-11|
|2016-01-12|
|2016-02-01|
|2016-02-02|
|2016-02-04|
|2016-02-05|
|2016-02-07|
|2016-02-08|
|2016-02-09|
|2016-02-11|
|2016-02-12|
+----------+
only showing top 20 rows



In [20]:
table_df.display()

AttributeError: 'DataFrame' object has no attribute 'display'

## 6. Consultas Personalizadas

Espaço para suas próprias consultas:

In [ ]:
# Sua consulta aqui
# Exemplo: Filtrar dados por uma condição específica
# gold_df.filter(gold_df.QTD_CORRIDAS > 1000).show()

## 7. Visualização dos Dados

Diferentes formas de visualizar os dados no PySpark:

In [ ]:
# Usando show() - forma básica
print("\nMétodo 1 - show():")
table_df.show(5)

# Usando show() com mais opções
print("\nMétodo 2 - show() com mais opções:")
table_df.show(5, truncate=False, vertical=True)

# Convertendo para Pandas (útil para visualizações mais elaboradas)
print("\nMétodo 3 - Convertendo para Pandas:")
pandas_df = table_df.toPandas()
print(pandas_df.head())

# Mostrando estatísticas descritivas
print("\nMétodo 4 - Estatísticas descritivas:")
table_df.describe().show()

In [ ]:
# Usando SQL (outra forma de visualizar)
print("\nMétodo 5 - Usando SQL:")
spark.sql("""
SELECT 
    DT_REFE,
    QTD_CORRIDAS,
    TOTAL_PASSAGEIROS
FROM info_corridas_do_dia
LIMIT 5
""").show()